# Prerequisites

In [ ]:
# Requirements
!pip install numpy matplotlib scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Definitions
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
rc('animation', html='jshtml')

from matplotlib.animation import FuncAnimation


def animate_trajectory(trajectory, resolution_Hz=3, duration=None, fig_size=10):
    plt.ioff()
    trajectory = np.array(trajectory)
    if len(trajectory.shape) == 1:
        trajectory = trajectory.reshape(trajectory.size, 1)
    if trajectory.shape == (trajectory.size, 1):
        trajectory = np.stack((trajectory.T[0], np.zeros(trajectory.size))).T
    if duration == None:
        frames = range(trajectory.shape[0])
    else:
        frames = range(int(duration * resolution_Hz))
    fig, ax = plt.subplots(figsize=(fig_size, fig_size))
    # set the axes limits
    ax.axis([-2,2,-2, 2])
    ax.set_aspect("equal")
    # create a point in the axes
    plt.grid()
    point, = ax.plot(0,1, marker="o")

    # Updating function, to be repeatedly called by the animation
    def update(t):
        # obtain point coordinates 
        x,y = trajectory[int(t) % trajectory.shape[0]]
        # set point's coordinates
        point.set_data([x],[y])
        return point,

    
    ani = FuncAnimation(fig, update, interval=1000/resolution_Hz, blit=True, repeat=True,
                    frames=frames)
    plt.ion()
    return ani


In [ ]:
x = np.array([1, -1]*10)
animate_trajectory(x)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


# Discrete-time systems

## Problem 1 

Compute and animate a trajectory of the following system (50 steps):

$x_{t + 1} := 0.9x_t, \ x\in \mathbb{R}$

$x_0 = 1$




In [ ]:
### YOUR CODE

animate_trajectory(trajectory)

## Problem 2 

Compute and animate a trajectory of the following system (300 steps, 20 Hz):

$x_{t + 1} := 0.95x_t + 0.05\xi_t, \ x_t\in \mathbb{R}, \ \xi_t \sim \mathcal{U}([-1, 1])$

$x_0 = 1$




In [ ]:
### YOUR CODE


## Discrete control-system interface 
A typical discrete control system would look something like this:

$x_{t + 1} = f(x_t, u_t), \ x_t \in \mathbb{X}, \ u_t \in \mathbb{U} := \{u \in \mathbb{R}^n \ | \ \lVert u \rVert_2 \leq r \},$

$u_t := \rho(x_t)$, 

$f(\cdot, \cdot)$ -- state transition function

$\rho(\cdot)$ -- feedback policy.

$x_0$ -- initial state.

Below is the implementation of a helper class designed to keep your modeling organized.




In [ ]:
from scipy.linalg import norm

class DiscreteTimeSystem:
    def __init__(self, 
                 state_transition_function,  # f(., .)
                 initial_state,              # x_0
                 max_norm_of_control=1       # r
                 ):
      self.__state_transition_function = state_transition_function
      self.__initial_state = initial_state
      self.__max_norm_of_control = max_norm_of_control

    def run_controller(self, 
                       feedback_policy,      # \rho
                       steps=100):
      trajectory = [self.__initial_state]
      for _ in range(steps):
        current_state = trajectory[-1]
        control_input = feedback_policy(current_state)
        assert norm(control_input) <= self.__max_norm_of_control, "Your control input does not belong to the control set. Fix your feedback."
        try:
            next_state = self.__state_transition_function(current_state, control_input)
        except OverflowError:
            print("The trjectory blew up. Ending the episode prematurely.")
            return np.array(trajectory)
        trajectory.append(next_state)
      return np.array(trajectory)

        


### Example
$x_{t+1}=x_t + u_t,\ x_t \in \mathbb{R}^2, \ u_t \in \mathbb{U} := \{u \in \mathbb{R}^2 \ | \ \lVert u \rVert_2 \leq 0.1 \},$

$x_0 = (1, 1),$

$u_t := \rho(x_t),$

$\rho(x) = (-0.05, -0.05)$



In [ ]:
def state_transition(state,  # x
                     control # u
                    ):
  return  state + control

initial_state = np.array([1, 1]) # x_0

system = DiscreteTimeSystem(state_transition_function=state_transition,
                            initial_state=initial_state,
                            max_norm_of_control=0.1)

def feedback(state):  # \rho
  return np.array([-0.05, -0.05])

trajectory = system.run_controller(feedback, steps = 50)

animate_trajectory(trajectory, resolution_Hz=10)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


## Problem 3
Manually implement a controller that drives the sytem's state to the origin in 100 steps:

$x_{t + 1} := x_t + u_t, \ x_t\in \mathbb{R}^2, \ u_t \in \mathbb{U} := \{u \in \mathbb{R}^2 \ | \ \lVert u \rVert_2 \leq 0.1 \},$

$x_0 = (1, 1)$

In [ ]:
from scipy.linalg import norm

def state_transition(state,  # x
                     control # u
                    ):
  return  state + control

initial_state = np.array([1, 1]) # x_0

system_3 = DiscreteTimeSystem(state_transition_function=state_transition,
                            initial_state=initial_state,
                            max_norm_of_control=0.09)

In [ ]:
### YOUR CODE


## Problem 4
Manually implement a controller that drives the sytem's state to the origin in a 100 steps:

$x_{t + 1} := x_t + u_t\lVert x_t - (\frac{1}{2}, 0)\rVert^2_2, \ x_t\in \mathbb{R}^2, \ u_t \in \mathbb{U} := \{u \in \mathbb{R}^2 \ | \ \lVert u \rVert_2 \leq 0.1 \},$

$x_0 = (1, 0)$

In [ ]:
from scipy.linalg import norm

def state_transition(state,  # x
                     control # u
                    ):
  return  state + control * norm(state - np.array([0.5, 0]))**2 

initial_state = np.array([1, 0]) # x_0

system_4 = DiscreteTimeSystem(state_transition_function=state_transition,
                            initial_state=initial_state,
                            max_norm_of_control=0.1)

In [ ]:
### YOUR CODE


## Problem 5
Let the feedback $\rho_\theta(\cdot)$ be implemented by a decision tree with parameters $\theta$. Implement an anlgorithm that optimizes the decision tree in such a way that it drives the system to the origin in 200 steps.

$x_{t + 1} := x_t + u_t\lVert x_t - (\frac{1}{2}, 0)\rVert^2_2, \ x_t\in \mathbb{R}^2, \ u_t \in \mathbb{U} := \{u \in \mathbb{R}^2 \ | \ \lVert u \rVert_2 \leq 0.1 \},$

$x_0 = (1, 0)$

$u_t := \rho_\theta(x_t), \ \theta \in \Theta := [-1, 1]^{15}$

In [ ]:
from scipy.linalg import norm

def state_transition(state,  # x
                     control # u
                    ):
  return  state + control * norm(state - np.array([0.5, 0]))**2 

initial_state = np.array([1, 0]) # x_0

system_5 = DiscreteTimeSystem(state_transition_function=state_transition,
                            initial_state=initial_state,
                            max_norm_of_control=0.1)

class DecisionTreeFeedback:
  def __init__(self, parameters=np.array([-1] * 15)):
    self.__parameters = parameters

  def __call__(self, state):
    if state[1] < self.__parameters[0]*2:
      if state[0] < self.__parameters[1]*2:
          if state[1] < self.__parameters[2]*2:
            return np.array([0.09, 0]) * self.__parameters[7]
          else:
            return np.array([0, 0.09]) * self.__parameters[8]
      else:
          if state[1] < self.__parameters[3]*2:
            return np.array([0.09, 0]) * self.__parameters[9]
          else:
            return np.array([0, 0.09]) * self.__parameters[10]
    else:
      if state[0] < self.__parameters[4]*2:
          if state[1] < self.__parameters[5]*2:
            return np.array([0.09, 0]) * self.__parameters[11]
          else:
            return np.array([0, 0.09]) * self.__parameters[12]
      else:
          if state[1] < self.__parameters[6]*2:
            return np.array([0.09, 0]) * self.__parameters[13]
          else:
            return np.array([0, 0.09]) * self.__parameters[14]


### What it does by default

In [ ]:
feedback = DecisionTreeFeedback()
trajectory = system_5.run_controller(feedback, steps = 100)

animate_trajectory(trajectory, resolution_Hz=10)

The trjectory blew up. Ending the episode prematurely.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


In [ ]:
### YOUR CODE
